In [1]:
import os
import zipfile
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ImportError: cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (C:\Users\Sandhya Ramachandra\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\framework\dtypes.py)

In [2]:
loaded_model = keras.models.load_model('cnn_model.keras')

NameError: name 'keras' is not defined

In [3]:
def predict_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    if img is not None:
        # Preprocess the image
        img_resized = cv2.resize(img, (224, 224))
        img_normalized = img_resized / 255.0
        img_input = np.expand_dims(img_normalized, axis=0)
        # Make prediction
        prediction = loaded_model.predict(img_input)
        probability = prediction[0][0]
        label = 'Bullying' if probability >= 0.5 else 'Non-Bullying'
        print(f"Prediction: {label}")
    else:
        print("Image not found or unable to read.")

In [4]:
# Example image paths (update with actual paths to your test images)
test_image_bullying = 'D:/ganesh/4-2/Front-end/dataset/bullying/1.jpg'
test_image_non_bullying = 'D:/ganesh/4-2/Front-end/dataset/non_bullying/255.jpg'

# Predict on a bullying image
predict_image(test_image_bullying)

# Predict on a non-bullying image
predict_image(test_image_non_bullying)

Image not found or unable to read.
Image not found or unable to read.


In [5]:
# ------------------------ Prediction Functions ------------------------

# Load the BERT model and tokenizer
import tensorflow as tf 
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bullying_detection_tf_model')

# Load the model
model = TFBertForSequenceClassification.from_pretrained('bullying_detection_tf_model')
# Function to predict text input
def predict_text(text):
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='tf'
    )

    outputs = model(encoding)
    logits = outputs.logits
    probs = tf.nn.softmax(logits, axis=1)
    pred = tf.argmax(probs, axis=1).numpy()[0]
    label_map = {0: 'Bullying', 1: 'Non-Bullying'}
    print(f"Prediction: {label_map[pred]} ")



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bullying_detection_tf_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [6]:
import streamlit as st
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification
import cv2
import numpy as np
import gradio as gr

# Load the BERT tokenizer and model for text prediction
bert_tokenizer = BertTokenizer.from_pretrained('bullying_detection_tf_model')
bert_model = TFBertForSequenceClassification.from_pretrained('bullying_detection_tf_model')

# Load the CNN model for image prediction
loaded_model = keras.models.load_model('cnn_model.keras')

# Function to predict text input
def predict_text(text):
    encoding = bert_tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='tf'
    )
    outputs = bert_model(**encoding)
    logits = outputs.logits
    probs = tf.nn.softmax(logits, axis=1)
    pred = tf.argmax(probs, axis=1).numpy()[0]
    label_map = {0: 'Bullying', 1: 'Non-Bullying'}
    prediction = label_map[pred]
    confidence = probs.numpy()[0][pred]
    return prediction, confidence

# Function to predict image input
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img_resized = cv2.resize(image, (224, 224))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)
    preds = loaded_model.predict(img_input)
    prob = preds[0][0]
    label = 'Bullying' if prob >= 0.5 else 'Non-Bullying'
    return label, prob

Some layers from the model checkpoint at bullying_detection_tf_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bullying_detection_tf_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [9]:

# Define Gradio interfaces
def predict_text_gradio(text):
    prediction, confidence = predict_text(text)
    return f"Prediction: {prediction}"

def predict_image_gradio(image):
    prediction, probability = predict_image(image)
    return f"Prediction: {prediction}"

# Create Gradio interfaces
text_interface = gr.Interface(
    fn=predict_text_gradio,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here..."),
    outputs="text",
    title="Digital Defender  - Text Input"
)

image_interface = gr.Interface(
    fn=predict_image_gradio,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Digital Defender  - Image Input"
)

# Combine interfaces into tabs
demo = gr.TabbedInterface([text_interface, image_interface], ["Text Input", "Image Input"])

# Launch the app
if __name__ == "__main__":
    demo.launch(share='true')

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://1ae7db499900194d0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\Sandhya Ramachandra\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sandhya Ramachandra\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sandhya Ramachandra\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sandhya Ramachandra\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
